<a href="https://colab.research.google.com/github/menpente/lenguaje_claro_cyc_2/blob/main/fine_tuning/litgpt_ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install litgpt

PENDIENTE: PREPARAR CUSTOM DATASET SIGUIENDO UN FORMATO SIMILAR A ESTE


[
  {
    "instruction": "Translate the following English text to French.",
    "input": "Hello, how are you?",
    "output": "Bonjour, comment ça va?"
  },
  {
    "instruction": "Summarize the following article.",
    "input": "The article discusses the impact of climate change on polar bears...",
    "output": "The article highlights the severe impact of climate change on polar bear populations..."
  }
]


Podemos crear el dataset en Excel o similar y luego utilizar pandas para guardar como json. El fichero debe llamarse my_custom_dataset.json

In [ ]:
#entrenamos el modelo con nuestros datos
!litgpt finetune microsoft/phi-2 \
  --data JSON \
  --data.json_path my_custom_dataset.json \
  --data.val_split_fraction 0.1 \
  --out_dir out/custom-model

In [ ]:
#lo probamos
!litgpt chat out/custom-model/final